In [9]:
%matplotlib inline

<html>
    <h1>Libraries used :</h1>
    <ul style="list-style-type: none;">
        <li>
             <img width=150px height=150px  src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Sympy_logo.svg/1200px-Sympy_logo.svg.png"  alt="Sympy">
        </li>
        <br>
        <li>
             <img width=300px height=300px  src="https://matplotlib.org/_static/logo2.png" alt="Matplotlib">
        </li>
        <br>
        <li>
             <img width=200px height=200px src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/NumPy_logo.svg/1200px-NumPy_logo.svg.png" alt="Numpy">
        </li>
        <br>
        <li>        
            <img width=200px height=200px  src="https://www.fullstackpython.com/img/logos/scipy.png" class="rounded" alt="Scipy">
        </li>
        <br>
        <li>
             <img width=400px height=400px  src="https://pandas.pydata.org/_static/pandas_logo.png" class="rounded" alt="Pandas">
        </li>
      </ul>

In [10]:
import numpy as np
import sympy as sm
from scipy.sparse import diags
from scipy.integrate import RK45, solve_ivp
import scipy.sparse as sp
import scipy.sparse.linalg as la
import matplotlib.pyplot as plt
import matplotlib


import time
import os
from functools import lru_cache
import math
import pandas as pd
from tqdm import tqdm_notebook


In [11]:
sm.init_printing()

plt.ioff()
plt.clf()

<html>
    <h3 align="center" style="color:#FF1C0C;"><u> Heat Equation </u> </h3> 
   <br>
</html>

\begin{equation}
\\\ u_t  + \beta u_{xx}    = - \left(\pi^{2} \beta - 1\right) e^{t} \sin{\left(\pi x \right)}
\\ u^h = \sum_{i= 0}^{n} a_i(t)  \phi_i(x) 
\\ \int_0^1 { (w u_t + \beta w u_{xx})} \ dx= \int_0^1 w ({- \left(\pi^{2} \beta - 1\right) e^{t} \sin{\left(\pi x \right)}}) dx
\\\ \int_0^1  w u_t   \ dx   - \beta \int_0^1 {w_x u_x} \ dx  + \beta \ (w(1) u_x(1) - w(0) u_x(0))= \int_0^1 w ({- \left(\pi^{2} \beta - 1\right) e^{t} \sin{\left(\pi x \right)}}) dx
\\  A \alpha^{'}(t) \hspace{5mm} \ -  \hspace{5mm}\beta  D \alpha(t) \hspace{10mm} +\hspace{10mm}  \beta E \alpha(t)   = f(t)
\\ w = {\phi_j}(x)
\end{equation}

\begin{equation}
\\  \alpha^{'}(t) = A^{-1}(\beta ( D - E) \alpha(t) + f(t)) \hspace{10mm} 
\end{equation}

<html>
    <h3 align="center" style="color:#FF1C0C;"><u>Boundary Conditions</u> </h3> <br>
</html>
\begin{equation}
\\  u(0, t) = 0  
\\  u(N, t) = 0
\\  u(x, 0) = sin(\pi x) 
\end{equation}

<html>
    <h3 align="center" style="color:#FF1C0C;"><u>Exact Solution</u> </h3> <br>
</html>
\begin{equation}
\\  u(x, t) = e^t sin (\pi x) 
\end{equation}

In [12]:
class Matrices:
    
    #A *alpha - beta(D *) +beta(E(1 - 0)alpha) = 0
    
    def __init__(self, N=10):
        
        self.N = N
        self.shape = (N+1, N+1)
        self.h = 1/N
        self.domain = np.linspace(0, 1, N+1) 
        self.beta = -1
        self.time_step = self.h**2
        self.runs = self.N**2
        
        self.A = (self.h) * sp.csc_matrix(diags([       [1/6 for i in range(N)],   \
                                        [1/3] + [2/3]*(N-1) + [1/3], \
                                             [1/6 for i in range(N)] ], \
                                                   [1, 0, -1]), dtype=np.float32)
        #-
        self.D = (1/self.h) * sp.csr_matrix(diags([      [-1 for i in range(N)],  \
                                      [1] + [2]*(N-1) + [1], \
                                        [-1 for i in range(N)]],  \
                             [1, 0, -1] ), dtype=np.float32)
                #+                
        row = np.array([0, 0, N, N])
        col = np.array([0, 1, N-1, N])
        data = np.array([1, -1, -1, 1 ])
        self.E = (1/self.h) *sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)    
      

        self.f_without_t = np.array(
            [(-(math.pi*self.h - math.sin(math.pi*self.h))*(math.pi**2*self.beta-1)/(math.pi**2*self.h))]+\
            [(2*(math.pi**2*self.beta - 1)*(math.cos(math.pi*self.h) - 1)*math.sin(math.pi*self.h*k)/(math.pi**2*self.h))\
            for k in range(1, self.N)]+[((math.pi**2*self.beta - 1)*(math.pi*self.h*math.cos(math.pi*self.N*self.h) - \
            math.sin(math.pi*self.N*self.h) + math.sin(math.pi*self.N*self.h - math.pi*self.h))/(math.pi**2*self.h))]).reshape(self.N+1, 1)
        

        self.f_t = lambda t:math.exp(t)*self.f_without_t
     

   
        #Set initial conditions here
        self.alpha = np.sin(np.pi* self.domain).reshape(N+1, 1)
        
        #Set solution here
        self.exact_solution = lambda t: np.sin(np.pi*self.domain).reshape(N+1, 1).dot(np.exp(t).reshape(1, -1))

In [13]:
class Methods(Matrices): 
    
    def __init__(self, num_basis = 10, method="Adam"):
        
        super().__init__(num_basis)
        eval("self."+method+"()")
        
        self.approx_half, self.approx_one = np.split(self.algo.y.flatten("F"), len(self.algo.t))  

        self.exact_half, self.exact_one =\
                            np.split(self.exact_solution(self.algo.t).flatten("F"), len(self.algo.t))

        self.error_half = math.sqrt(sum((self.approx_half - self.exact_half)**2))
        self.error_one = math.sqrt(sum((self.approx_one - self.exact_one)**2))
        
    

        
    def fun(self, t, y):

        y[0] = 0
        y[-1] = 0

        return la.inv(self.A).dot(self.beta*(self.D - self.E).dot(y) + self.f_t(t))
    
    def Adam(self):   
        self.algo = solve_ivp(self.fun,t_span=(0, 1),  y0=self.alpha.ravel(), \
                     t_eval= [0.5, 1], vectorized=True, first_step = self.h, method="LSODA")
        self.algo.y[[0, -1], :] = 0 # refresh boundary conditions
        
        

    def RK45(self):
        pass

In [14]:
heateqn = Methods( method="Adam")

In [ ]:
points = [5*j for j in range(3, 30)]

half = pd.DataFrame(data =None, columns=["Approximate", "Exact", "$ ||.||_2 \ Error $"])
one = pd.DataFrame(data =None, columns=["Approximate", "Exact", "$ ||.||_2 \ Error $"])

for i in tqdm_notebook(points):
    heateqn = Methods(i, method="Adam")
    half.loc[i] = [heateqn.approx_half, heateqn.exact_half, heateqn.error_half]
    one.loc[i] = [heateqn.approx_one, heateqn.exact_one, heateqn.error_one]
    
    
    

In [ ]:
if ~os.path.exists("Heat Equation t=.5 Data.csv"):
    half.to_csv("Heat Equation t=.5 Data.csv")
if ~os.path.exists("Heat Equation t=1 Data.csv"):
    one.to_csv("Heat Equation t=1 Data.csv")

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)